<a href="https://colab.research.google.com/github/rajivsam/cmi_count_data_modeling/blob/asif_ikbal_21_07_2021/customers_per_hour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import pandas as pd
import numpy as np

In [65]:
## Loading the data
path='https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
data=pd.read_excel(path)

In [66]:
## CustomerID is loaded as float. Changing it to int
data['CustomerID'] = data['CustomerID'].apply(lambda x: int(x) if x == x else "")

In [67]:
## Creating Date and Hour columns from the timestamp InvoiceDate
## Here Hour is the truncated time of the day. if the time is hh:mm:ss, Hour is hh only.
data['Date']=[d.date() for d in data['InvoiceDate']]
data['Hour']=[d.hour for d in data['InvoiceDate']]

In [68]:
#data.head(10)

In [69]:
## Columns of our interest
dateTimeData=data[['CustomerID','Date','Hour']]

In [70]:
## Counting number of customers in a particular Date and Hour
table=dateTimeData.groupby(['Date','Hour']).agg({"CustomerID": "nunique"})

In [71]:
## Converting grouped object to DataFrame
tab2=pd.DataFrame(table.reset_index())

In [72]:
#tab2.head()

In [73]:
## Creating the required DataFrame with columns Date, Count1, Count2,...Count24.
## Counti represents the time between hour i-1(inclusive) and i(exclusive)
colNames=['Date']
colNames.extend(['Count'+str(k+1) for k in range(24)])
newDf=pd.DataFrame(columns=colNames)
for date in set(tab2['Date']):
  tab3=tab2[tab2['Date']==date][['Hour','CustomerID']]
  newRow=[date]
  newRow.extend([0 for k in range(24)])
  for hr in tab3['Hour']:
    newRow[int(hr)+1]=int(tab3[tab3['Hour']==hr]['CustomerID'])
  #print(pd.DataFrame([newRow],columns=colNames))
  newDf=newDf.append(pd.DataFrame([newRow],columns=colNames),ignore_index=True)


In [74]:
newDf=newDf.sort_values(by=['Date']).reset_index()

In [75]:
newDf=newDf.drop('index',axis=1)

In [76]:
newDf.to_csv('customers_per_hour.csv')